## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kyabe,TD,9.45,18.94,98.33,15,68,5.50,broken clouds
1,1,Monrovia,LR,6.30,-10.80,86.00,70,75,3.36,broken clouds
2,2,Gasa,PS,31.50,34.47,70.00,53,0,3.00,clear sky
3,3,Busselton,AU,-33.65,115.33,52.00,67,11,5.99,few clouds
4,4,Westview,US,25.88,-80.24,73.00,88,20,5.82,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Monrovia,LR,6.30,-10.80,86.00,70,75,3.36,broken clouds
2,2,Gasa,PS,31.50,34.47,70.00,53,0,3.00,clear sky
4,4,Westview,US,25.88,-80.24,73.00,88,20,5.82,few clouds
5,5,New Norfolk,AU,-42.78,147.06,66.20,39,0,12.75,clear sky
11,11,Dharmabad,IN,18.90,77.85,81.00,52,33,5.53,scattered clouds
13,13,Codrington,AU,-38.27,141.97,64.49,72,100,6.20,light rain
14,14,Chuy,UY,-33.70,-53.46,66.04,62,28,14.61,scattered clouds
15,15,Aden,YE,12.78,45.04,79.83,72,27,13.78,scattered clouds
18,18,Rikitea,PF,-23.12,-134.97,75.16,81,67,11.18,broken clouds
26,26,Port Elizabeth,ZA,-33.92,25.57,71.60,60,0,33.33,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                344
City                   344
Country                341
Lat                    344
Lng                    344
Max Temp               344
Humidity               344
Cloudiness             344
Wind Speed             344
Current Description    344
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Monrovia,LR,6.30,-10.80,86.00,70,75,3.36,broken clouds
2,2,Gasa,PS,31.50,34.47,70.00,53,0,3.00,clear sky
4,4,Westview,US,25.88,-80.24,73.00,88,20,5.82,few clouds
5,5,New Norfolk,AU,-42.78,147.06,66.20,39,0,12.75,clear sky
11,11,Dharmabad,IN,18.90,77.85,81.00,52,33,5.53,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
704,704,Akyab,MM,20.15,92.90,78.46,73,16,10.65,few clouds
712,712,Jeremie,HT,18.65,-74.12,76.41,74,0,2.08,clear sky
714,714,Netivot,IL,31.42,34.59,70.00,52,0,3.00,clear sky
720,720,Broken Hill,AU,-31.95,141.43,87.80,18,0,3.36,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Monrovia,LR,86.00,broken clouds,6.30,-10.80,
2,Gasa,PS,70.00,clear sky,31.50,34.47,
4,Westview,US,73.00,few clouds,25.88,-80.24,
5,New Norfolk,AU,66.20,clear sky,-42.78,147.06,
11,Dharmabad,IN,81.00,scattered clouds,18.90,77.85,
13,Codrington,AU,64.49,light rain,-38.27,141.97,
14,Chuy,UY,66.04,scattered clouds,-33.70,-53.46,
15,Aden,YE,79.83,scattered clouds,12.78,45.04,
18,Rikitea,PF,75.16,broken clouds,-23.12,-134.97,
26,Port Elizabeth,ZA,71.60,clear sky,-33.92,25.57,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Monrovia,LR,86.00,broken clouds,6.30,-10.80,Eco-Hotel Hall & Restaurant
2,Gasa,PS,70.00,clear sky,31.50,34.47,رابطة الفنانين الفلسطينيين
4,Westview,US,73.00,few clouds,25.88,-80.24,MiLuSol Hostel
5,New Norfolk,AU,66.20,clear sky,-42.78,147.06,The Shingles Riverside Cottages
11,Dharmabad,IN,81.00,scattered clouds,18.90,77.85,LBS College's Boys Hostel
...,...,...,...,...,...,...,...
704,Akyab,MM,78.46,few clouds,20.15,92.90,Hotel Memory
712,Jeremie,HT,76.41,clear sky,18.65,-74.12,Auberge’Inn
714,Netivot,IL,70.00,clear sky,31.42,34.59,Catering Gato - kosher catering Dairy
720,Broken Hill,AU,87.80,clear sky,-31.95,141.43,Daydream Motel & Cottages


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))